<a href="https://colab.research.google.com/github/Seb85vickz/AI-projects/blob/main/Chatgpt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, Dense, LayerNormalization, Dropout
from tensorflow.keras import Model

In [2]:
class MultiHeadAttention(Layer):
  def __init__(self, embed_dim, num_heads):
    super().__init__()
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    self.attn_head_size = embed_dim // num_heads

    self.wq = Dense(embed_dim)
    self.wk = Dense(embed_dim)
    self.wv = Dense(embed_dim)

    self.dense = Dense(embed_dim)
  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.attn_head_size))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self,q,k,v,mask):
    batch_size = tf.shape(q)[0]

    q = self.split_heads(self.wq(q), batch_size)
    k = self.split_heads(self.wk(k), batch_size)
    v = self.split_heads(self.wv(v), batch_size)

    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
      scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    output = tf.transpose(output, perm=[0, 2, 1, 3])
    concat_attention = tf.reshape(output, (batch_size, -1, self.embed_dim))
    return self.dense(concat_attention)

In [3]:
class FeedForwardNetwork(Layer):
  def __init__(self,embed_dim,diff):
    super().__init__()
    self.dense1 = Dense(diff, activation='gelu')
    self.dense2 = Dense(embed_dim)

  def call(self,x):
    return self.dense2(self.dense1(x))